# PixelMapVars Reader

Simple notebook to read PixelMapVars from art/LArSoft ROOT files using uproot.
This handles the custom art format used by SBNDPDSProducer.

In [76]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [77]:
# File path - update this to your pixelmap_variables.root location
file_path = "/exp/sbnd/app/users/svidales/larsoft_develop/run_try/tenth_attemp/pixelmap_variables.root"

print(f"Reading file: {file_path}")
print(f"File exists: {Path(file_path).exists()}")
print(f"File size: {Path(file_path).stat().st_size / 1024:.1f} KB")

Reading file: /exp/sbnd/app/users/svidales/larsoft_develop/run_try/tenth_attemp/pixelmap_variables.root
File exists: True
File size: 478.3 KB


In [78]:
# Open file and explore structure
file = uproot.open(file_path)

print(">> ROOT file contents:")
for key in file.keys():
    print(f"  - {key}")

# Focus on Events tree
events_tree = file['Events']
print(f"\n>> Events tree has {events_tree.num_entries} entries")
print(">> Branches in Events tree:")
for branch in events_tree.keys():
    if 'PixelMapVars' in branch:
        print(f"  * {branch}")
    else:
        print(f"  - {branch}")

>> ROOT file contents:
  - RootFileDB;1
  - MetaData;1
  - FileIndex;1
  - Parentage;1
  - Events;1
  - EventMetaData;1
  - SubRuns;1
  - SubRunMetaData;1
  - Runs;1
  - RunMetaData;1
  - ResultsTree;1
  - ResultsMetaData;1

>> Events tree has 22 entries
>> Branches in Events tree:
  - EventAuxiliary
  * PixelMapVars_opanatree__SBNDPDSProd.
  * PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.present
  * PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.rangeSetID
  * PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.obj


In [79]:
# Lectura directa de las 4 branches de PixelMapVars
print("=== Lectura directa de branches PixelMapVars ===")

pixelmap_branches = [
    'PixelMapVars_opanatree__SBNDPDSProd.',
    'PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.present',
    'PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.rangeSetID',
    'PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.obj'
]

branch_data = {}

for branch_name in pixelmap_branches:
    print(f"\\n--- {branch_name} ---")
    
    # Probar diferentes métodos de lectura
    for library in ['ak', 'np']:
        try:
            data = events_tree[branch_name].array(library=library)
            branch_data[branch_name] = {'data': data, 'library': library}
            
            print(f"✅ {library}: Éxito!")
            print(f"  Tipo: {type(data)}")
            print(f"  Longitud: {len(data)}")
            
            # Mostrar algunos valores de muestra
            if hasattr(data, 'to_list'):
                sample = data.to_list()[:3]
            else:
                sample = data[:3] if len(data) > 0 else "Sin datos"
            print(f"  Muestra (3 primeros): {sample}")
            
            # Si es un array estructurado, mostrar detalles
            if hasattr(data, 'dtype') and hasattr(data.dtype, 'names') and data.dtype.names:
                print(f"  Campos: {data.dtype.names}")
            
            break  # Usar el primer método exitoso
            
        except Exception as e:
            print(f"❌ {library}: {type(e).__name__} - {str(e)[:80]}")

print(f"\\n=== Resumen ===")
print(f"Branches leídas exitosamente: {len(branch_data)}/4")

=== Lectura directa de branches PixelMapVars ===
\n--- PixelMapVars_opanatree__SBNDPDSProd. ---
❌ ak: AssertionError - 
✅ np: Éxito!
  Tipo: <class 'dict'>
  Longitud: 3
❌ np: TypeError - unhashable type: 'slice'
\n--- PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.present ---
✅ ak: Éxito!
  Tipo: <class 'awkward.highlevel.Array'>
  Longitud: 22
  Muestra (3 primeros): [True, True, True]
\n--- PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.rangeSetID ---
✅ ak: Éxito!
  Tipo: <class 'awkward.highlevel.Array'>
  Longitud: 22
  Muestra (3 primeros): [4294967295, 4294967295, 4294967295]
\n--- PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.obj ---
❌ ak: AssertionError - 
✅ np: Éxito!
  Tipo: <class 'numpy.ndarray'>
  Longitud: 22
  Muestra (3 primeros): [<PixelMapVars (version 1) at 0x7f8704367400>
 <PixelMapVars (version 1) at 0x7f8704367460>
 <PixelMapVars (version 1) at 0x7f8704367ee0>]
\n=== Resumen ===
Branch

In [83]:
# Inspección específica de los objetos PixelMapVars
print("=== Inspección de objetos PixelMapVars ===")

obj_branch = 'PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.obj'

if obj_branch in branch_data:
    obj_data = branch_data[obj_branch]['data']
    print(f"Tenemos {len(obj_data)} objetos PixelMapVars")
    
    # Examinar los primeros objetos en detalle
    for i in range(min(3, len(obj_data))):
        pixelmap_obj = obj_data[i]
        print(f"\\n=== Objeto PixelMapVars {i} ===")
        print(f"Tipo: {type(pixelmap_obj)}")
        print(f"Representación: {repr(pixelmap_obj)}")
        
        # Intentar acceder a atributos comunes de PixelMapVars
        print("\\nAtributos disponibles:")
        attributes = dir(pixelmap_obj)
        
        # Filtrar atributos relevantes (coordenadas)
        coord_attributes = []
        for attr in attributes:
            if not attr.startswith('_'):  # Omitir atributos privados
                coord_attributes.append(attr)
        
        print(f"Atributos públicos: {coord_attributes}")
        
        # Intentar acceder a cada atributo
        print("\\nValores de atributos:")
        for attr in coord_attributes:
            try:
                value = getattr(pixelmap_obj, attr)
                if callable(value):
                    print(f"  {attr}(): método")
                else:
                    print(f"  {attr}: {value} (tipo: {type(value)})")
            except Exception as e:
                print(f"  {attr}: Error - {e}")
        
        # Buscar específicamente atributos de coordenadas
        print("\\nBúsqueda específica de coordenadas:")
        coord_names = ['true_x', 'true_y', 'true_z', 'pred_x', 'pred_y', 'pred_z',
                      'diff_x', 'diff_y', 'diff_z', 'truth_x', 'truth_y', 'truth_z',
                      'predicted_x', 'predicted_y', 'predicted_z', 'reco_x', 'reco_y', 'reco_z']
        
        found_coords = {}
        for coord_name in coord_names:
            if hasattr(pixelmap_obj, coord_name):
                try:
                    value = getattr(pixelmap_obj, coord_name)
                    found_coords[coord_name] = value
                    print(f"  ✅ {coord_name}: {value}")
                except Exception as e:
                    print(f"  ❌ {coord_name}: Error - {e}")
        
        if found_coords:
            print(f"\\n🎯 Coordenadas encontradas: {found_coords}")
        else:
            print("\\n❌ No se encontraron atributos de coordenadas con nombres esperados")
            
            # Intentar métodos alternativos
            print("\\nProbando métodos alternativos...")
            
            # ¿Tiene un método para obtener coordenadas?
            for method_name in ['getCoordinates', 'getTruePos', 'getPredPos', 'getPosition', 'coordinates']:
                if hasattr(pixelmap_obj, method_name):
                    try:
                        result = getattr(pixelmap_obj, method_name)()
                        print(f"  ✅ {method_name}(): {result}")
                    except Exception as e:
                        print(f"  ❌ {method_name}(): Error - {e}")
            
            # ¿Es iterable?
            try:
                if hasattr(pixelmap_obj, '__iter__') or hasattr(pixelmap_obj, '__getitem__'):
                    print(f"  Probando acceso como secuencia...")
                    for j in range(10):  # Probar los primeros 10 índices
                        try:
                            value = pixelmap_obj[j]
                            print(f"    [{j}]: {value}")
                        except (IndexError, KeyError, TypeError):
                            break
                        except Exception as e:
                            print(f"    [{j}]: Error - {e}")
                            break
            except Exception as e:
                print(f"  No es iterable: {e}")

else:
    print("❌ No se pudieron obtener los objetos PixelMapVars")

=== Inspección de objetos PixelMapVars ===
Tenemos 22 objetos PixelMapVars
\n=== Objeto PixelMapVars 0 ===
Tipo: <class 'uproot.dynamic.Model_PixelMapVars_v1'>
Representación: <PixelMapVars (version 1) at 0x7f8704367400>
\nAtributos disponibles:
Atributos públicos: ['all_members', 'awkward_form', 'base', 'base_names_versions', 'bases', 'behaviors', 'check_numbytes', 'class_code', 'class_flags', 'class_rawstreamers', 'class_streamer', 'class_version', 'classname', 'close', 'closed', 'concrete', 'cursor', 'empty', 'encoded_classname', 'file', 'has_member', 'hook_after_read_members', 'hook_before_postprocess', 'hook_before_read', 'hook_before_read_members', 'instance_version', 'is_instance', 'is_memberwise', 'member', 'member_names', 'members', 'num_bytes', 'parent', 'postprocess', 'read', 'read_member_n', 'read_members', 'read_numbytes_version', 'serialize', 'strided_interpretation', 'to_pyroot', 'to_writable', 'tojson', 'writable']
\nValores de atributos:
  all_members: {'flash_ophit_pe

In [84]:
# ¡ÉXITO! Extracción de coordenadas usando el atributo 'members'
print("=== 🎯 Extracción exitosa de coordenadas ===")

obj_branch = 'PixelMapVars_opanatree__SBNDPDSProd./PixelMapVars_opanatree__SBNDPDSProd.obj'

if obj_branch in branch_data:
    obj_data = branch_data[obj_branch]['data']
    
    coordinate_data = {
        'true_x': [], 'true_y': [], 'true_z': [],
        'pred_x': [], 'pred_y': [], 'pred_z': [],
        'diff_x': [], 'diff_y': [], 'diff_z': []
    }
    
    successful_extractions = 0
    
    for event_idx, pixelmap_obj in enumerate(obj_data):
        try:
            # Acceder al diccionario 'members' que contiene las coordenadas
            members = pixelmap_obj.members
            
            # Verificar que el objeto tiene datos (no está vacío)
            if hasattr(members['dEpromx'], '__len__') and len(members['dEpromx']) > 0:
                # Extraer coordenadas verdaderas
                true_x = float(members['dEpromx'][0])  # Primer (y único) elemento del vector
                true_y = float(members['dEpromy'][0])
                true_z = float(members['dEpromz'][0])
                
                # Extraer coordenadas predichas
                pred_x = float(members['dEpromx_pred'][0])
                pred_y = float(members['dEpromy_pred'][0])
                pred_z = float(members['dEpromz_pred'][0])
                
                # Extraer diferencias
                diff_x = float(members['dEpromx_diff'][0])
                diff_y = float(members['dEpromy_diff'][0])
                diff_z = float(members['dEpromz_diff'][0])
                
                # Agregar a los datos
                coordinate_data['true_x'].append(true_x)
                coordinate_data['true_y'].append(true_y)
                coordinate_data['true_z'].append(true_z)
                coordinate_data['pred_x'].append(pred_x)
                coordinate_data['pred_y'].append(pred_y)
                coordinate_data['pred_z'].append(pred_z)
                coordinate_data['diff_x'].append(diff_x)
                coordinate_data['diff_y'].append(diff_y)
                coordinate_data['diff_z'].append(diff_z)
                
                successful_extractions += 1
                
                print(f"Evento {event_idx}: ✅")
                print(f"  True: ({true_x:.1f}, {true_y:.1f}, {true_z:.1f})")
                print(f"  Pred: ({pred_x:.1f}, {pred_y:.1f}, {pred_z:.1f})")
                print(f"  Diff: ({diff_x:.1f}, {diff_y:.1f}, {diff_z:.1f})")
            else:
                print(f"Evento {event_idx}: ❌ (sin datos)")
                
        except Exception as e:
            print(f"Evento {event_idx}: ❌ Error - {e}")
    
    print(f"\\n📊 Resumen de extracción:")
    print(f"✅ Eventos exitosos: {successful_extractions}/{len(obj_data)}")
    print(f"📈 Tasa de éxito: {100*successful_extractions/len(obj_data):.1f}%")

else:
    print("❌ No se pudo acceder a los datos del branch .obj")
    successful_extractions = 0

=== 🎯 Extracción exitosa de coordenadas ===
Evento 0: ❌ (sin datos)
Evento 1: ✅
  True: (146.0, 171.6, 48.1)
  Pred: (143.3, 178.2, 63.2)
  Diff: (-2.7, 6.6, 15.1)
Evento 2: ❌ (sin datos)
Evento 3: ❌ (sin datos)
Evento 4: ❌ (sin datos)
Evento 5: ✅
  True: (115.7, 168.2, 438.5)
  Pred: (122.7, 179.3, 448.8)
  Diff: (7.0, 11.1, 10.3)
Evento 6: ✅
  True: (-138.3, -55.6, 76.1)
  Pred: (147.0, -54.0, 73.9)
  Diff: (8.6, 1.6, -2.2)
Evento 7: ❌ (sin datos)
Evento 8: ✅
  True: (8.0, -187.9, 499.5)
  Pred: (34.0, -188.9, 487.0)
  Diff: (26.0, -0.9, -12.5)
Evento 9: ✅
  True: (-194.6, -198.0, 312.4)
  Pred: (189.6, -190.4, 324.1)
  Diff: (-5.0, 7.6, 11.7)
Evento 10: ✅
  True: (-193.7, -181.5, 373.4)
  Pred: (194.5, -191.3, 382.3)
  Diff: (0.8, -9.8, 8.9)
Evento 11: ✅
  True: (-95.1, -179.7, 147.6)
  Pred: (86.0, -181.5, 158.1)
  Diff: (-9.0, -1.8, 10.5)
Evento 12: ❌ (sin datos)
Evento 13: ❌ (sin datos)
Evento 14: ✅
  True: (163.1, -90.5, 123.4)
  Pred: (159.4, -82.2, 112.7)
  Diff: (-3.7, 8.2, -

In [85]:
# Crear la tabla de resultados detallada
if successful_extractions > 0:
    n_events = len(coordinate_data['true_x'])
    
    # Crear DataFrame
    results_data = []
    for i in range(n_events):
        results_data.append({
            'Event': i,
            'True_X': coordinate_data['true_x'][i],
            'Pred_X': coordinate_data['pred_x'][i],
            'Diff_X': coordinate_data['diff_x'][i],
            'True_Y': coordinate_data['true_y'][i],
            'Pred_Y': coordinate_data['pred_y'][i],
            'Diff_Y': coordinate_data['diff_y'][i],
            'True_Z': coordinate_data['true_z'][i],
            'Pred_Z': coordinate_data['pred_z'][i],
            'Diff_Z': coordinate_data['diff_z'][i]
        })
    
    df_results = pd.DataFrame(results_data)
    
    # Mostrar la tabla con el formato exacto que pediste
    print(f"\\n>> Detailed Results Table ({n_events} Test Events):")
    print("=" * 100)
    print("* Showing all events:")
    
    # Configurar formato
    pd.set_option('display.float_format', '{:.1f}'.format)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 200)
    pd.set_option('display.max_rows', None)
    
    # Mostrar la tabla
    print(df_results.to_string(index=False))
    
    # Resumen de rendimiento
    print(f"\\n>> Performance Summary ({n_events} events):")
    print("=" * 40)
    
    coord_names = ['X', 'Y', 'Z']
    for coord in coord_names:
        diff_values = df_results[f'Diff_{coord}'].values
        print(f"* {coord} coordinate:")
        print(f"  - Mean difference (Bias): {np.mean(diff_values):6.2f} cm     [mean(diff)]")
        print(f"  - Std difference:         {np.std(diff_values):6.2f} cm     [sqrt(mean((diff - mean(diff))**2))]")
        print(f"  - RMSE:                   {np.sqrt(np.mean(diff_values**2)):6.2f} cm     [sqrt(mean(diff**2))]")
        print(f"  - MAE:                    {np.mean(np.abs(diff_values)):6.2f} cm     [mean(|diff|)]")
        print()
    
    # RMSE general
    overall_rmse_per_coord = []
    for coord in coord_names:
        diff_values = df_results[f'Diff_{coord}'].values
        rmse = np.sqrt(np.mean(diff_values**2))
        overall_rmse_per_coord.append(rmse)
    
    overall_rmse = np.sqrt(np.mean(np.array(overall_rmse_per_coord)**2))
    print(f">> Overall 3D RMSE: {overall_rmse:.2f} cm")
    
    # Rangos de coordenadas
    print(f"\\n>> Coordinate Ranges:")
    for coord in coord_names:
        true_vals = df_results[f'True_{coord}'].values
        pred_vals = df_results[f'Pred_{coord}'].values
        print(f"* {coord} coordinate:")
        print(f"  - True range: [{np.min(true_vals):.1f}, {np.max(true_vals):.1f}] cm")
        print(f"  - Pred range: [{np.min(pred_vals):.1f}, {np.max(pred_vals):.1f}] cm")
    
    print(f"\\n🎉 ¡Éxito total! Se extrajeron y mostraron {n_events} eventos de PixelMapVars!")
    
else:
    print("\\n❌ No se extrajo ningún evento con coordenadas.")
    print("Verifica que el archivo ROOT contenga eventos con datos.")

# Cerrar archivo
file.close()
print("\\n📁 Archivo cerrado.")

\n>> Detailed Results Table (12 Test Events):
* Showing all events:
 Event  True_X  Pred_X  Diff_X  True_Y  Pred_Y  Diff_Y  True_Z  Pred_Z  Diff_Z
     0   146.0   143.3    -2.7   171.6   178.2     6.6    48.1    63.2    15.1
     1   115.7   122.7     7.0   168.2   179.3    11.1   438.5   448.8    10.3
     2  -138.3   147.0     8.6   -55.6   -54.0     1.6    76.1    73.9    -2.2
     3     8.0    34.0    26.0  -187.9  -188.9    -0.9   499.5   487.0   -12.5
     4  -194.6   189.6    -5.0  -198.0  -190.4     7.6   312.4   324.1    11.7
     5  -193.7   194.5     0.8  -181.5  -191.3    -9.8   373.4   382.3     8.9
     6   -95.1    86.0    -9.0  -179.7  -181.5    -1.8   147.6   158.1    10.5
     7   163.1   159.4    -3.7   -90.5   -82.2     8.2   123.4   112.7   -10.6
     8   147.9   142.5    -5.4    78.6    72.5    -6.1   210.3   225.5    15.2
     9  -108.3   108.2    -0.1   -33.7   -30.8     2.9    28.3    35.1     6.8
    10   148.4   145.1    -3.3  -180.1  -184.3    -4.3   482.0 

In [ ]:
# 🔍 DIAGNÓSTICO: Investigar el problema del signo en True_X
print("=== 🔍 Diagnóstico del problema de signo en True_X ===")

if obj_branch in branch_data and successful_extractions > 0:
    obj_data = branch_data[obj_branch]['data']
    
    print("\\nComparando los valores tal como se guardan vs. como se procesan:")
    print("-" * 80)
    
    for event_idx, pixelmap_obj in enumerate(obj_data):
        try:
            members = pixelmap_obj.members
            
            # Solo eventos con datos
            if hasattr(members['dEpromx'], '__len__') and len(members['dEpromx']) > 0:
                # Valor original tal como se guarda
                original_true_x = float(members['dEpromx'][0])
                original_true_y = float(members['dEpromy'][0])
                original_true_z = float(members['dEpromz'][0])
                
                # Como se procesaría en el notebook de inferencia (valor absoluto para X)
                processed_true_x = abs(original_true_x)  # ← Aplicar valor absoluto
                processed_true_y = original_true_y       # ← Y se mantiene igual
                processed_true_z = original_true_z       # ← Z se mantiene igual
                
                print(f"Evento {event_idx}:")
                print(f"  True_X original:   {original_true_x:8.1f}   |   Procesado (abs): {processed_true_x:8.1f}")
                print(f"  True_Y:            {original_true_y:8.1f}   |   (sin cambio)")
                print(f"  True_Z:            {original_true_z:8.1f}   |   (sin cambio)")
                
                # Mostrar predicciones para comparar
                pred_x = float(members['dEpromx_pred'][0])
                pred_y = float(members['dEpromy_pred'][0])
                pred_z = float(members['dEpromz_pred'][0])
                
                print(f"  Pred_X:            {pred_x:8.1f}")
                print(f"  Pred_Y:            {pred_y:8.1f}")
                print(f"  Pred_Z:            {pred_z:8.1f}")
                
                # Verificar si las predicciones están en rango positivo (como se espera)
                if pred_x >= 0:
                    print("  ✅ Pred_X es positivo (como se espera para coordenadas absolutas)")
                else:
                    print("  ⚠️ Pred_X es negativo (inesperado)")
                    
                print()
                
        except Exception as e:
            print(f"Evento {event_idx}: Error - {e}")
    
    print("\\n🔍 ANÁLISIS:")
    print("1. El módulo LArSoft guarda True_X con el signo original (puede ser negativo)")
    print("2. El notebook de inferencia aplica abs() a True_X antes del entrenamiento")
    print("3. El modelo CNN fue entrenado solo con valores X positivos (abs)")
    print("4. Las predicciones son siempre positivas")
    print("\\n💡 SOLUCIÓN: Aplicar valor absoluto a True_X para consistencia")

else:
    print("No hay datos para diagnosticar")